In [1]:
model_type = 'gpt' #phi
dataset_type = 'sqa' #gsm8k #aqua
prompt_type = 'baseline' #cot #qap
params = {'token':400,'qap':50}
output_file = ''


#Installation

In [3]:
!pip install datasets
if model_type == 'phi':
  !pip install transformers
  !pip install flash-attn
  !pip install einops
#!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from datasets import load_dataset
import json
dataset = load_dataset("wics/strategy-qa")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public

Generating test split: 0 examples [00:00, ? examples/s]

In [7]:
dataset['test'][250]["answer"]

True

In [ ]:
if model_type == 'phi':
  import torch
  from transformers import AutoModelForCausalLM, AutoTokenizer

  torch.set_default_device("cuda")

  tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)
  model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1_5:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/33.5k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1_5:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

In [5]:
from torch.utils.data import DataLoader
import re

def extract_last_answer(full_text):
  try:
    segments = full_text.split("A: ")
    last_answer = segments[-1].strip()
    last_answer = last_answer.split("Question 1:")[0].strip()
    return last_answer

  except ValueError as e:
    return None

def extract_output_numerical_answer(text, key_phrase="The answer is"): # You may need to create variations of this function to extract your intended numerical answer, depending on the output of your prompt.
    if text is None:
        return None

    try:
        # Find the position where 'The answer is' starts in the text
        start_pos = text.index(key_phrase) + len(key_phrase)

        # Extract the text that comes after 'The answer is'
        answer_text = text[start_pos:].strip()

        # Use a regular expression to find a floating point or integer number
        matches = re.findall(r'[-+]?\d*\.\d+|\d+', answer_text)

        if matches:
            return float(matches[0])
        else:
            return None

    except ValueError:
        return None

In [ ]:
strat_cot_prompts = """Q: Would a soccer player practice with a baseball?
A: A soccer player would practice with a soccer ball. Baseballs are used by baseball players. #False
Q: Would Elon Musk play a leading role in a Marvel movie?
A: Elon Musk is a businessman. He does not usually act in movies. He does not have a degree in acting. He is not known for being an actor. #False
Q: Is Barcelona a rival soccer club to Real Madrid?
A: Barcelona vs. Real Madrid is called the El Clasico. They are the two best teams in LaLiga. They draw in many viewers. #True
Q: Were tanks used in the American Revolution?
A: Tanks were invented in the early twentieth century. The American Revolution ended in 1783. #False
Q: Would you buy clothes at Target?
A: Target is a popular retail store. They consist of many items including clothes. #True
Q: Would the sun become a black hole?
A: The sun is an average-sized star. It is not big enough to become a black-hole. It will become a white dwarf. #False
Q: Is Michael Phelps an Olympic Champion?
A: Michael Phelps has won 28 total Olympic medals. He is the most decorated Olympic athlete in history. #True
Q: Would Apple sell shoes?
A: Apple is a famous technology company. They are known for their phones, airpods, iPads, and MacBooks. #False"""

In [ ]:
key = " " # enter your key here

In [12]:
!pip install openai==1.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.6.1
    Uninstalling openai-1.6.1:
      Successfully uninstalled openai-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [10]:
!pip install typing-extensions==4.5.0

  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
openai 1.6.1 requires typing-extensions<5,>=4.7, but you have typing-extensions 4.5.0 which is incompatible.


#GPT 3.5 Turbo

In [13]:
import openai
from openai import OpenAI
import re
import time

client = OpenAI(api_key=key)

correct = 0

#f = open("/content/drive/MyDrive/log/result.txt", "a")

#d = open("/content/drive/MyDrive/log/answers.txt", "a")

s,e = 0,99

with (open("/content/drive/MyDrive/WS_Full_StratQA_Log/GPT3.5_Turbo/PASP_Result.txt", "w") as f,
    open("/content/drive/MyDrive/WS_Full_StratQA_Log/GPT3.5_Turbo/PASP_Answer.txt", "w") as d):

  f.write("PASP without system prompt: \n")
  d.write("PASP without system prompt: \n")
  for i, batch in enumerate(dataset["test"]): # Change debug_dataloader to train_dataloader or dev_dataloader when necessary
    if i < s:
      continue
    if(i > e):
      break
    question = batch['question']
    #prompt = f"{question} Take a deep breath and work on this problem step-by-step."
    #prompt = f"{question} Explain this problem to me in at least 25 words. Then solve for the answer."
    #prompt = f"{question}\n{options}\n"
    #prompt = question
    #prompt = strat_cot_prompts + question
    prompt =  f"{question} Let's first understand the problem, extract relevant variables and their corresponding numerals, " \
             "and devise a complete plan. Then, let's carry out the plan, calculate intermediate variables " \
             "(pay attention to correct numerical calculation and commonsense), " \
             "solve the problem step by step, and show the answer."
    true_answer = batch['answer']
    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    temperature = 0,
    seed=42,
    messages=[
      #{"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": prompt}
             ]
    )
    answer_line = response.choices[0].message.content
    our_answer = re.findall("(True|False)", answer_line,re.IGNORECASE)
    if our_answer == []:
      our_answer = "[]"
    else:
      our_answer = our_answer[-1]
    d.write(f"Question[{i}]:\n {question}\n")
    d.write(f"Correct answer:\n {true_answer}\n")
    d.write(f"Full answer:\n {answer_line}\n")
    d.write(f"Our answer:\n {our_answer}\n")
    d.write("-"*100+"\n")
    if our_answer and our_answer == str(true_answer):
      correct += 1
      f.write(f"{i} Correct \n")
      print(f"{i} Correct")
    else:
      f.write(f"{i} Incorrect \n")
      print(f"{i} Incorrect")
    d.flush()
    f.flush()
    #if (i + 1) % 3 == 0:
    #  time.sleep(60)
  score = (correct/(i))*100
  print(score)
  f.write(f"Score: {score}")

ImportError: cannot import name 'is_list_type' from 'openai._utils._utils' (/usr/local/lib/python3.10/dist-packages/openai/_utils/_utils.py)

In [ ]:
our_answer == str(true_answer)

True

#Phi Model

In [ ]:
def extract_CoT_sqa(full_answer, question):
  temp = full_answer.split(question)
  if(len(temp)) > 1:
    ans = temp[1].split("Q:")
    return ans[0]
  else:
    return temp


In [ ]:
import time

with (open("/content/drive/MyDrive/StratQA_Log/Phi/PASP_Result.txt", "a") as f,
  open("/content/drive/MyDrive/StratQA_Log/Phi/PASP_Answer.txt", "a") as d):

  f.write("PASP+. : \n")
  d.write("PASP+. : \n")
  s, e = 0, 100
  correct = 0
  for i, batch in enumerate(dataset["test"]): # Change debug_dataloader to train_dataloader or dev_dataloader when necessary
    if i < s:
      continue
    if(i > e):
      break
    question = batch['question']
    prompt = f"{question} Take a deep breath and work on this problem step-by-step."
    #prompt = f"{question} Explain this problem to me in at least 25 words. Then solve for the answer."
    #prompt = f"{question}\n{options}\n"
    #prompt = f"{question} Explain this problem to me in at least 200 words. Then solve for the answer."
    #prompt = f"Explain this problem to me in at least 50 words. Then solve for the answer. {question}"
    #prompt = f"{question} Take a deep breath and work on this problem step-by-step."
    #prompt = question
    #prompt = f"{strat_cot_prompts}\nQ: {question}"
    #prompt = question_analysis_prompt_few_shot + "\n" + "Q: " + question
    #prompt = QAP_COT_Combination + "\n" + "Q: " + question
    prompt = f"{question} Let's first understand the problem, extract relevant variables and their corresponding numerals, " \
         "and devise a complete plan. Then, let's carry out the plan, calculate intermediate variables " \
         "(pay attention to correct numerical calculation and commonsense), " \
         "solve the problem step by step, and show the answer."
    true_answer = batch['answer']
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = inputs.to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=350, do_sample=False) # You may need to modify max_new_tokens

    answer_line = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    answer_line = extract_CoT_sqa(answer_line, question)
    our_answer = re.findall("#(True|False)", answer_line,re.IGNORECASE)
    if our_answer == []:
      our_answer = "[]"
    else:
      our_answer = our_answer[0]
    d.write(f"Question[{i}]:\n {question}\n")
    d.write(f"Correct answer:\n {true_answer}\n")
    d.write(f"Full answer:\n {answer_line}\n")
    d.write(f"Our answer:\n {our_answer}\n")
    d.write("-"*100+"\n")
    if our_answer and our_answer == str(true_answer):
      correct += 1
      f.write(f"{i} Correct \n")
      print(f"{i} Correct")
    else:
      f.write(f"{i} Incorrect \n")
      print(f"{i} Incorrect")
    d.flush()
    f.flush()
  score = (correct/(i))*100
  print(score)
  f.write(f"Score: {score}")


0 Incorrect
1 Incorrect
2 Incorrect
3 Incorrect
4 Incorrect
5 Incorrect
6 Incorrect
7 Incorrect
8 Incorrect
9 Incorrect
10 Incorrect
11 Incorrect
12 Incorrect
13 Incorrect
14 Incorrect
15 Incorrect
16 Incorrect
17 Incorrect
18 Incorrect
19 Incorrect
20 Incorrect
21 Incorrect
22 Incorrect
23 Incorrect
24 Incorrect
25 Incorrect
26 Incorrect
27 Incorrect
28 Incorrect
29 Incorrect
30 Incorrect
31 Incorrect
32 Incorrect
33 Incorrect
34 Incorrect
35 Incorrect
36 Incorrect
37 Incorrect
38 Incorrect
39 Incorrect
40 Incorrect
41 Incorrect
42 Incorrect
43 Incorrect
44 Incorrect
45 Incorrect
46 Incorrect
47 Incorrect
48 Incorrect
49 Incorrect
50 Incorrect
51 Incorrect
52 Incorrect
53 Incorrect
54 Incorrect
55 Incorrect
56 Incorrect
57 Incorrect
58 Incorrect
59 Incorrect
60 Incorrect
61 Incorrect
62 Incorrect
63 Incorrect
64 Incorrect
65 Incorrect
66 Incorrect
67 Incorrect
68 Incorrect
69 Incorrect
70 Incorrect
71 Incorrect
72 Incorrect
73 Incorrect
74 Incorrect
75 Incorrect
76 Incorrect
77 Incorr

In [ ]:
temp = answer_line.split(question)

In [ ]:
answer = temp[1].split("Q:")

In [ ]:
answer[0].

'\nA: Yes, more people today are related to Genghis Khan than Julius Caesar. Genghis Khan was a Mongol leader who conquered much of Asia. Julius Caesar was a Roman emperor who died young. #True\n'